In [ ]:
import numpy
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets
from torch.utils.data import DataLoader

In [ ]:
dataset = datasets.MNIST('.', download=True)

In [ ]:
dataset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x7F77BB660ED0>, 5)

In [ ]:
import matplotlib.pyplot as plt
noized = dataset.data[1234].float() / 255 + torch.normal(
    torch.zeros_like(dataset.data[1234].float()), 
    0.2 * torch.ones_like(dataset.data[1234].float()))
plt.imshow(noized.detach().numpy())
plt.show()

In [ ]:
# hyperparams
in_chan = 1
#TODO change model h_params
hidden_ch = 64
out_ch = 1
device_id = 0
device = 'cpu' if device_id == -1 else f'cuda:{device_id}'
n_epochs = 10
batch_size = 128
noise_factor = 0.4

In [ ]:
class Encoder(nn.Module):
  #28x28 -> 14x14 -> 7x7 
  def __init__(self, in_chan, hidden_ch, out_ch):
    super().__init__()
    #TODO modify architecture as you wish. Add more layers, make hidden smaller, etc
    pass

  def forward(self, x):
    pass

    return x

class Decoder(nn.Module):
  #hidden_out-> hidden -> 28*28 
  def __init__(self, in_chan, hidden_ch, out_ch):
    super().__init__()
    #TODO modify architecture as you wish. Add more layers, make hidden smaller, etc
    pass

  def forward(self, x):
    pass

    return x

class ConvAutoEncoder(nn.Module):
  def __init__(self, in_chan, hidden_ch, out_ch):
    super().__init__()
    self.encoder = Encoder(in_chan, hidden_ch, out_ch)
    self.decoder = Decoder(in_chan, hidden_ch, out_ch)

  def forward(self, x):
    hidden = self.encoder(x)
    x_ = self.decoder(hidden)

    return x_


In [ ]:
def collate_fn_conv(data: list):
  # data = [(pic, target)...]
  pics = []
  target = []
  for item in data:
    pics.append(numpy.array(item[0]))
    target.append(item[1])
  pics = torch.from_numpy(numpy.array(pics)).float() / 255 # B x W x H
  target = torch.from_numpy(numpy.array(target))

  return {
      'data': pics.unsqueeze(1), # B x 1 x W x H
      'target': target.long(),
      }

In [ ]:
model_conv = ConvAutoEncoder(in_chan, hidden_ch, out_ch).to(device)
optim = torch.optim.Adam(model_conv.parameters())
loss_func = nn.MSELoss()

In [ ]:
for epoch in range(n_epochs):
  dataloader = DataLoader(dataset, 
                          batch_size, 
                          shuffle=True, 
                          collate_fn=collate_fn_conv,
                          drop_last = True,
                          )
  for i, batch in enumerate(dataloader):
    optim.zero_grad()
    data = batch['data'].to(device)
    noized = torch.clamp(data + 
                         torch.normal(torch.zeros_like(data), 
                                      noise_factor * torch.ones_like(data)), 0., 1.)
    predict = model_conv(noized)
    loss = loss_func(predict, data)
    loss.backward()
    optim.step()
    if i % 200 == 0:
      print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
  item = dataset.data[1234].unsqueeze(0).unsqueeze(0).float()/255
  item = torch.clamp(item + 
                     torch.normal(torch.zeros_like(item), 
                                  noise_factor * torch.ones_like(item)), 0., 1.)
  plt.imshow(item.squeeze().cpu().detach().numpy())
  plt.show()
  plt.imshow(model_conv(item.to(device)).squeeze().cpu().detach().numpy())
  plt.show()
  torch.save(model_conv.state_dict(), f'./conv_ae_chkpt_conv_{epoch}.pth')